In [1]:
from ultralytics import YOLO
import cv2
import easyocr
import re

In [2]:
model = YOLO(r'Models\best.pt')
reader = easyocr.Reader(['en'])

In [3]:
def process_image(image_path):
    frame = cv2.imread(image_path)
    resized_img = cv2.resize(frame, (640, 480))
    results = model.predict(resized_img)

    output_data = []

    for r in results:
        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0]
            plate_roi = resized_img[int(b[1]):int(b[3]), int(b[0]):int(b[2])]

            number_plate_results = reader.readtext(plate_roi)
            number_plate_text = ''
            for result in number_plate_results:
                text = result[1]

                text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
                number_plate_text += text + ' '


            number_plate_text = number_plate_text.replace(" ", "").upper()  
            number_plate_text = re.sub(r'[^a-zA-Z0-9]', '', number_plate_text)  


            expected_format = re.compile(r'^[A-Z0-9]{1,10}$')
            if expected_format.match(number_plate_text):


                    output_data.append({
                        "Plate Coordinates": list(map(int, b)),
                        "Number Plate Text": number_plate_text.strip()
                    })

    #print("Output Data:", output_data)
    return output_data

In [11]:
image_path = r'Car Images\Test Image 8.jpg'
process_image(image_path)


0: 480x640 1 0, 134.7ms
Speed: 8.0ms preprocess, 134.7ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)


[{'Plate Coordinates': [316, 340, 410, 391], 'Number Plate Text': 'AFR2012'}]